In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


from input_generator import load_dataset, load_dataset_with_lables
from data_parser import get_tags_and_labels, get_vocab, get_tags
from tag_handler import create_seperate_tags, create_listed_inputs

d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [3]:
# Get dataframe with tags and category_id columns
csvpath = 'data/CAvideos.cvs'
tab_dataframe = get_tags_and_labels(csvpath)
max_of_tags = 124
# tab_dataframe = create_seperate_tags(tab_dataframe, max_of_tags)
tab_dataframe = create_listed_inputs(tab_dataframe)

tab_dataframe.head()

KeyError: 'tag'

In [ ]:
# Getting leaning vocab
vocab = get_vocab(get_tags(csvpath))
vocab_size = len(vocab)

In [ ]:
voc_df = pd.DataFrame(columns=np.arange(vocab_size+1))

voc_df.rename(columns={i:'tags{}'.format(i-1) for i in range(1, vocab_size)},
                 inplace=True)
voc_df.rename(columns={0:'category_id'},
                 inplace=True)

voc_df.head()

In [ ]:
voc_di = {i: vo for i, vo in enumerate(vocab)}
voc_di[0] = 'notags'
voc_di = {vo: i for i, vo in voc_di.items()}

# Mapping tags to indexes in vocab
for index in range(0, max_of_tags):
    tab_dataframe['tags{}'.format(index)] = tab_dataframe['tags{}'.format(index)].map(voc_di)

tab_dataframe.fillna(0.0, inplace = True) 
tab_dataframe.head()


In [ ]:
for row in tab_dataframe.rows:
    for data in row:
        

In [ ]:
# Create train,validation and test examples from dataframe
# train_ds, val_ds, test_ds = load_dataset(tab_dataframe)
(train_ds, train_lb), (val_ds, val_lb), (test_ds, test_lb) = load_dataset_with_lables(tab_dataframe, 'category_id')


In [ ]:
# first 5 training examples
print(train_ds.values[5:10])
train_lb.values[0:5]

In [ ]:
hidden = 32
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(max_of_tags,)),
    layers.Dense(hidden,activation='relu'),
    layers.Dropout(0.7),
    layers.Dense(2*hidden,activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.7),
    layers.Dense(44, activation='softmax', kernel_regularizer=keras.regularizers.l2(0.01))])

model.compile(optimizer='nadam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', 'sparse_categorical_crossentropy'],
              )

history = model.fit(train_ds.values, train_lb.values,
                    batch_size=64,
                    epochs=100,
                    validation_data=(val_ds, val_lb))
